In [ ]:
import numpy as np
import pandas as pd
import os
import json
import re
import math
import warnings

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
df_dianoil = pd.read_csv('../data/JD_energy_ratio.csv')
display(df_dianoil.head())

In [ ]:
# Match waybill_base with the tram fuel car ratio for 3 months
df_result = pd.read_csv('../data/waybill_base_20240910.csv')

with open('../data/city_name_list.json', 'r') as f:
    city_name_list = json.load(f)

month_list = ['2023_01','2023_07','2024_01']
for month in month_list:
    df_result[f'elevator_ratio_{month}'] = 0

for i in range(len(df_result)):
    city_result = df_result.loc[i,'city']
    flag = False
    repeat_list = []
    for temp_list in city_name_list:
        if city_result in temp_list:
            repeat_list = temp_list
            for city_temp in repeat_list:
                if city_temp in df_dianoil['city'].values:
                    for month in month_list:
                        elevator_ratio = df_dianoil.loc[df_dianoil['city']==city_temp,f'elevator_ratio_{month}'].values
                        elevator_ratio = re.sub(r'%', '', elevator_ratio[0])
                        elevator_ratio = float(elevator_ratio) / 100
                        df_result.loc[df_result['city']==city_result,f'elevator_ratio_{month}'] = elevator_ratio
                        flag = True
    if len(repeat_list) == 0:
        if  city_result in df_dianoil['city'].values:
            for month in month_list:
                elevator_ratio = df_dianoil.loc[df_dianoil['city']==city_result,f'elevator_ratio_{month}'].values
                elevator_ratio = re.sub(r'%', '', elevator_ratio[0])
                elevator_ratio = float(elevator_ratio) / 100
                df_result.loc[df_result['city']==city_result,f'elevator_ratio_{month}'] = elevator_ratio
                flag = True

    if not flag:
        print(f'error：{city_result}')

city_list = ['Ngari Prefecture', 'Yushu Tibetan Autonomous Prefecture', 'Tiemenguan City', 'Shuanghe City', 'Nagqu City', 'Kokdala City', 'Huyanghe City']
for month in month_list:
    df_temp = df_result[df_result['city_level']=='Tire5']
    df_temp = df_temp.dropna(subset=[f'elevator_ratio_{month}'])
    mean_ratio = df_temp[f'elevator_ratio_{month}'].mean()
    df_result.loc[df_result['city'].isin(city_list),f'elevator_ratio_{month}'] = mean_ratio

df_result.to_csv('../data/waybill_base_20240910.csv',index=False)

In [ ]:
# Count the number of employees of each company in each city every month
with open('../data/city_name_list.json', 'r') as f:
    city_name_list = json.load(f)

df_result = pd.read_csv('../data/waybill_base_20240910.csv')

company_list = ['JD','SF','YT','YD','ZT','ST']
month_list = ['2023_01','2023_07','2024_01']

for month in month_list:
    print(month)
    for company in company_list:
        filepath = f'../../{company}/final_result/{company}_final_result_{month}.csv'
        df_result[f'num_operator_{company}_{month}'] = 0
        if not os.path.exists(filepath):
            continue
        print(company)
        df_company = pd.read_csv(filepath,low_memory=False)

        # data filtering
        data = df_company['all_distance'].to_numpy()
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_company = df_company[(df_company['all_distance']>lower_bound)&(df_company['all_distance']<upper_bound)]

        print(f'sum operator num:{len(df_company)}')
        city_company_list = df_company['city'].unique().tolist()
        df_result[f'num_operator_{company}_{month}'] = 0
        
        for city in city_company_list:
            df_city = df_company[df_company['city'] == city].reset_index()
            
            flag = False
            repeat_list = []
            city_res = '' 
            for temp_list in city_name_list:
                if city in temp_list:
                    repeat_list = temp_list
                    for city_temp in repeat_list:
                        if city_temp in df_result['city'].values:
                            city_res = city_temp
                            flag = True
                            break
            if len(repeat_list) == 0:
                if city in df_result['city'].values:
                    city_res = city
                    flag = True
        
            if not flag:
                print(f'match error：{city}')
                continue

            df_result.loc[df_result['city']==city_res,f'num_operator_{company}_{month}'] = len(df_city)

df_result.to_csv('../data/waybill_base_20240911.csv',index=False)

In [ ]:
'''
Increase the number of packages from different companies in each city
'''

df_result = pd.read_csv('../data/waybill_base_20240911.csv')

city_exclude = ['China-North Korea Joint Area', 'Sansha City', 'Taiwan', 'Hong Kong', 'Macau', 'Shuanghe City']
company_list = ['JD','SF','YT','YD','ZT','ST']
month_list = ['2023_01','2023_07','2024_01']
pkg_num_dict = {'2023_01':{'JD':18550000,'SF':31900000,'YT':37366666,'YD':34333333,'ZT':44133333,'ST':26866666},
               '2023_07':{'JD':17230000,'SF':29433333,'YT':56333333,'YD':52033333,'ZT':78233333,'ST':47966666},
               '2024_01':{'JD':24580000,'SF':38233333,'YT':71700000,'YD':65233333,'ZT':94333333,'ST':60333333}}

for month in month_list:
    print(month)
    for company in company_list:
        print(company)
        df_result[f'num_package_{company}_{month}'] = 0
        pkg_sum = pkg_num_dict[month][company]
        for i in range(len(df_result)):
            city_name = df_result.loc[i,'city']
            if city_name not in city_exclude:
                ratio = df_result.loc[i,'ratio']
                pkg_num = 0
                if not ratio > 0:
                    pkg_num = 50
                else:
                    pkg_num = int(pkg_sum * ratio)
                df_result.loc[i, f'num_package_{company}_{month}'] = pkg_num
        
df_result.to_csv('../data/waybill_base_20240911.csv',index=False)

In [ ]:
'''
Calculating carbon emissions for three months and three particle sizes (New edition)
'''

with open('../data/city_name_list.json', 'r') as f:
    city_name_list = json.load(f)

car_weight,man_weight,pkg_weight = 350,69.6,1.38
coeff_ele = 0.000113  # One kilogram of carbon is emitted per kilogram of objects transported
coeff_oil = 0.15 # 1km of carbon emissions of oil vehicles kg
cm_weight = car_weight + man_weight

df_result = pd.read_csv('../data/waybill_base_20240911.csv')

company_list = ['JD','SF','YT','YD','ZT','ST']
month_list = ['2023_01','2023_07','2024_01']

for month in month_list:
    print(month)
    for company in company_list:
        # Initialize the value and complete it at the end
        df_result[f'sum_Ce_{company}_{month}'] = 0
        df_result[f'average_operator_Ce_{company}_{month}'] = 0
        df_result[f'average_package_Ce_{company}_{month}'] = 0
        
        filepath = f'../../{company}/final_result/{company}_final_result_{month}.csv'
        if not os.path.exists(filepath):
            continue
        print(company)
        df_company = pd.read_csv(filepath,low_memory=False)

        # Data filtering
        data = df_company['all_distance'].to_numpy()
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_company = df_company[(df_company['all_distance']>lower_bound)&(df_company['all_distance']<upper_bound)]
        
        city_company_list = df_company['city'].unique().tolist()
        
        for city in city_company_list:
            df_city = df_company[df_company['city'] == city].reset_index()
            
            # Look for the city name in waybill_base
            flag = False
            repeat_list = []
            city_res = '' 
            for temp_list in city_name_list:
                # Have the same name, go through the same name to the horse
                if city in temp_list:
                    repeat_list = temp_list
                    for city_temp in repeat_list:
                        if city_temp in df_result['city'].values:
                            city_res = city_temp
                            flag = True
                            break
            if len(repeat_list) == 0:
                if city in df_result['city'].values:
                    city_res = city
                    flag = True
        
            if not flag:
                print(f'match error：{city}')
                continue

            # We randomly assign energy to each guy in proportion to the trolley
            elevator_ratio = df_result.loc[df_result['city']==city_res,f'elevator_ratio_{month}'].values
            df_city['energy'] = np.random.rand(len(df_city)) # 0 electricity 1 oil
            df_city['energy'] = df_city['energy'].apply(lambda x: 1 if x > elevator_ratio else 0)

            pkg_num = df_result.loc[df_result['city']==city_res,f'num_package_{company}_{month}'].values[0]
            
            pkg_avg_operator = int(pkg_num/len(df_city)) # The number of packages in the city is higher than the number of small brothers
            if pkg_avg_operator < 1:
                pkg_avg_operator = 1
            
            # Calculate each guy's carbon footprint
            df_city['Ce'] = 0 # Carbon emission, unit kg
            df_city.loc[df_city['energy']==1,'Ce'] = df_city['all_distance'] * coeff_oil
            df_city.loc[df_city['energy']==0,'Ce'] = coeff_ele * (cm_weight * df_city['all_distance'] + pkg_weight * pkg_avg_operator * df_city['all_distance']/df_city['wave_count']/2)
            
            # Total emissions of city particle size
            city_sum_Ce = df_city['Ce'].sum()
            # The average emission of city particle size
            num_operator = df_result.loc[df_result['city']==city_res,f'num_operator_{company}_{month}'].values[0]
            city_average_operator_Ce = city_sum_Ce / num_operator
            # The average emission of city particle size parcels          
            city_average_package_Ce = city_sum_Ce / pkg_num
            
            df_result.loc[df_result['city']==city_res, f'sum_Ce_{company}_{month}'] = city_sum_Ce
            df_result.loc[df_result['city']==city_res, f'average_operator_Ce_{company}_{month}'] = city_average_operator_Ce
            df_result.loc[df_result['city']==city_res, f'average_package_Ce_{company}_{month}'] = city_average_package_Ce

df_result.to_csv('../data/sum_result_20240911.csv',index=False)

In [ ]:
'''
Calculate the distance of three months and three grains
'''
with open('../data/city_name_list.json', 'r') as f:
    city_name_list = json.load(f)

df_result = pd.read_csv('../data/sum_result_20240911.csv')
# df_result = pd.read_csv('../data/cooperator_sum_result_20240924.csv')

company_list = ['JD','SF','YT','YD','ZT','ST']
month_list = ['2023_01','2023_07','2024_01']

for month in month_list:
    print(month)
    for company in company_list:
        print(company)
        df_result[f'sum_distance_{company}_{month}'] = 0
        df_result[f'average_operator_distance_{company}_{month}'] = 0
        df_result[f'average_package_distance_{company}_{month}'] = 0
        filepath = f'../../{company}/final_result/{company}_final_result_{month}.csv'
        if not os.path.exists(filepath):
            continue
        df_company = pd.read_csv(filepath,low_memory=False)

        if company in ['SF','ZT','YT','YD']:
            df_company = df_company.drop_duplicates(subset=['courier','phone'])
        if company == 'JD':
            df_company = df_company.drop_duplicates(subset=['operator_id'])

        # data filtering
        data = df_company['all_distance'].to_numpy()
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_company = df_company[(df_company['all_distance']>lower_bound)&(df_company['all_distance']<upper_bound)]
        
        city_company_list = df_company['city'].unique().tolist()

        for city in city_company_list:
            df_city = df_company[df_company['city'] == city].reset_index()
    
            # Look for the city name in waybill_base
            flag = False
            repeat_list = []
            city_res = '' 
            for temp_list in city_name_list:
                if city in temp_list:
                    repeat_list = temp_list
                    for city_temp in repeat_list:
                        if city_temp in df_result['city'].values:
                            city_res = city_temp
                            flag = True
                            break
            if len(repeat_list) == 0:
                if city in df_result['city'].values:
                    city_res = city
                    flag = True
        
            if not flag:
                print(f'match error：{city}')
                continue
            
            # Total distance of city granularity
            city_sum_distance = df_city['all_distance'].sum()
            # The average distance of city granularity
            num_operator = df_result.loc[df_result['city']==city_res,f'num_operator_{company}_{month}'].values[0]
            city_average_operator_distance = city_sum_distance / num_operator
            pkg_num = df_result.loc[df_result['city']==city_res,f'num_package_{company}_{month}'].values[0]
            city_average_package_distance = city_sum_distance / pkg_num
            df_result.loc[df_result['city']==city_res, f'sum_distance_{company}_{month}'] = city_sum_distance
            df_result.loc[df_result['city']==city_res, f'average_operator_distance_{company}_{month}'] = city_average_operator_distance
            df_result.loc[df_result['city']==city_res, f'average_package_distance_{company}_{month}'] = city_average_package_distance

df_result.to_csv('../data/sum_result_20240911.csv',index=False)
# df_result.to_csv('temp_df/cooperator_sum_result_20240924.csv',index=False)